In [ ]:
%reload_ext autoreload
%autoreload 2

import gif
import pylab as plt
import numpy as np
from pythonperlin import perlin

def remove_margins():
    """ Removes figure margins, keeps only plot area """
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0,0)
    return


## Generate perlin noise

In [ ]:
%%time

""" Generate noise """
dens = 24
shape = (6,6)
p = perlin(shape, dens=dens, seed=0)

""" 
Use Perlin noise as a phase of a complex number z 
to generate a direction vector noise for x and y at each grid point.
"""
z = np.exp(2j * np.pi * p)
dx, dy = z.real, z.imag


## Draw flow field
<br>

- To generate a flowfield we use Perlin noise (no octaves) as a vector field. Value at each grid node gives a direction vector (gradient). Then drop random dots and calculate how they move along the gradients. 

In [ ]:
def plot_vector_field(z):
    """ Draws a vector field """
    m, n = z.shape
    for i in range(m):
        for j in range(n):
            di = 0.5 * z[i,j].real
            dj = 0.5 * z[i,j].imag
            plt.arrow(i, j, di, dj, color='grey', width=0.1, alpha=0.1)
    return
            
def calculate_flow_line(z, niter=96, seed=None):
    """
    Puts a random pint and calculates its flow line 
    in the vector field z for niter iterations.
    """
    np.random.seed(seed)
    m, n = z.shape
    # Initialize x, y for a line
    x = np.zeros((niter,2)) * np.nan
    # put a starting point at random location
    x[0] = z.shape * np.random.uniform(-0.1, 1.1, size=2)
    # For niter steps move a dot along the direction of
    # the closest vector of the vector field.
    for k in range(niter-1):
        x0 = x[k]
        i, j = x0.astype(int)
        i = i % m
        j = j % n
        g = .3 * np.array([z[i,j].real, z[i,j].imag])
        x[k+1] = x0 + g
    return x.T


def dimlight(i, niter=96):
    """ Dims the last frames before redrawing again from the start """
    alpha = 1 - 1 / (1 + np.exp(0.5 * (niter - i - 10)))
    return alpha

def plot_flow_field(i, x, z, color, show_vectors=False):
    xlim, ylim = z.shape
    alpha = 0.1 * dimlight(i)
    fig = plt.figure(figsize=(6,6), facecolor="black")
    remove_margins()
    for k, x_ in enumerate(x):
        plt.plot(x_[0][:i], x_[1][:i], color=color[k], lw=2, alpha=alpha)
    plt.xlim(0,xlim)
    plt.ylim(0,ylim)
    plt.tight_layout()
    return fig

""" Generate flow lines and random color for each line """
x = [calculate_flow_line(z, seed=None) for i in range(7200)]
color = plt.get_cmap("coolwarm")(np.random.uniform(0,1,len(x)))

""" Show one frame of the flow field """
plot_flow_field(16, x, z, color, show_vectors=True)
plt.savefig("flowfield.jpg")

## Animate flow field using line flow iteration as a time axis

In [ ]:
# Set the dots per inch resolution
gif.options.matplotlib["dpi"] = 180

# Decorate a plot function with @gif.frame
@gif.frame
def plot(i, x, z, color):
    plot_flow_field(i, x, z, color)

# Construct "frames"
niter = 96
frames = [plot(i, x, z, color) for i in range(niter)]

# Save "frames" to gif with a specified duration (milliseconds) between each frame
gif.save(frames, 'flowfield.gif', duration=100)

## FIN